In [12]:
import os
import yaml
import json
import pandas as pd
from pathlib import Path

In [2]:
# # # parser.add_argument('mode', type=str,
# # #     choices=['make-ome', 'segment-ome', 'generate-spatial-features', 'generate-region-features', show-channels'],
# # #     help='Which task mip is to execute.')

# ###############################
# ## generate-region-features ##
# ###############################
# # parser.add_argument('--input-tif', type=str,
# #     help='ome.tiff file to generate spatial features for')

# # parser.add_argument('--output-prefix', type=str, default='output',
# #     help='Output prefix to use when writing output files. Two files will be written: *_spatial_features.h5ad and *_spatial_features.txt. Default is "output". For example, if --output-prefix is "path/to/out/directory/sample" then the two output files will be named path/to/out/directory/sample_spatial_features.h5ad and path/to/out/directory/sample_spatial_features.txt.')

# parser.add_argument('--mask-tif', type=str,
#     help='Filepath of region mask tif.')

# parser.add_argument('--mask-markers', type=str, default='Pan-Cytokeratin,E-cadherin',
#     help='If --mask-tif is not provided, these markers will be used to generate a region mask.')

# parser.add_argument('--spatial-features', type=str,
#     help='Filepath to .h5ad file output by generate-spatial-features.')

In [3]:
cwl_dir = '/diskmnt/Projects/Users/estorrs/multiplex-imaging-pipeline/cwl'
Path(cwl_dir).mkdir(parents=True, exist_ok=True)

In [6]:
template = {
    'output_prefix': 'a_string',
    'input_tif': {
        'class': 'File',
        'path': 'a/file/path',
    },
    'mask_tif': {
        'class': 'File',
        'path': 'a/file/path',
    },
    'spatial_features': {
        'class': 'File',
        'path': 'a/file/path',
    },
    'mask_markers': 'a_string'
}

In [7]:
yaml.safe_dump(template, open(os.path.join(cwl_dir, 'template.region_features.yaml'), 'w'))

In [13]:
cwl = {
    'class': 'CommandLineTool',
    'cwlVersion': 'v1.0',
    '$namespaces': {'sbg': 'https://www.sevenbridges.com/'},
    'id': 'region_features',
    'label': 'region_features',
    'baseCommand': ['mip'],
    'inputs': [
        {
            'id': 'input_tif',
            'type': 'File',
            'inputBinding': {
                'position': '0',
                'prefix': '--input-tif',
            }
        },
        {
            'id': 'spatial_features',
            'type': 'File',
            'inputBinding': {
                'position': '0',
                'prefix': '--spatial-features',
            }
        },
        {
            'id': 'mask_tif',
            'type': 'File?',
            'inputBinding': {
                'position': '0',
                'prefix': '--mask-tif',
            }
        },
        {
            'id': 'mask_markers',
            'type': 'string?',
            'default': 'Pan-Cytokeratin,E-cadherin',
            'inputBinding': {
                'prefix': '--mask-markers',
                'position': '0'
            }
        },
        {
            'id': 'output_prefix',
            'type': 'string?',
            'default': 'output',
            'inputBinding': {
                'prefix': '--output-prefix',
                'position': '0'
            }
        },
        
        # needs path to be set so it works on compute1
        {
            'id': 'environ_PATH',
            'type': 'string?',
            'default': '/miniconda/envs/mip/bin:$PATH'
        },
    ],
    'arguments': [
        {
            'position': 1,
            'valueFrom': 'generate-region-features'
        },
    ],
    'outputs': [
        {
            'id': 'output_txt',
            'type': 'File',
            'outputBinding': {'glob': '*.txt'}
        },
        {
            'id': 'region_mask',
            'type': 'File',
            'outputBinding': {'glob': '*region_mask.tif'}
        },
    ],
    'requirements': [
        {
            'class': 'DockerRequirement',
            'dockerPull': 'estorrs/multiplex-imaging-pipeline:0.0.1'
        },
        {
            'class': 'ResourceRequirement',
            'ramMin': 100000
        },
        {
            'class': 'EnvVarRequirement',
            'envDef': {
                'PATH': '$(inputs.environ_PATH)',
            }
        }
    ]
}

In [14]:
yaml.safe_dump(cwl, open(os.path.join(cwl_dir, 'region_features.cwl'), 'w'))